In [1]:
from ktms import *

In [2]:
def modpos(sites):
    modsites = []
    for i in range(len(sites)):
        modsite = []
        for j in range(len(sites[i]) - 1):
            if sites[i][j] < 0:
                sites[i][j] += np.ceil(abs(sites[i][j]))
            elif sites[i][j] >= 1:
                sites[i][j] -= np.floor(sites[i][j])
            
            if abs(sites[i][j]) < 0.01 or abs(sites[i][j] - 1) < 0.01:
                sites[i][j] = 0
            modsite.append(sites[i][j])
        modsite.append(sites[i][2])
        modsites.append(modsite)
    return modsites

In [3]:
def checkifsame(bareatoms, sites, allused):
    '''
    Check if in allused has same configuration of symmetrical operated sites.
    Return True if found.
    '''
    struct = AseAtomsAdaptor.get_structure(bareatoms)
    surf_sg = SpacegroupAnalyzer(struct, 0.01)
    symm_ops = surf_sg.get_symmetry_operations()

    cpallused = copy.deepcopy(allused)
    frcpallused = []
    
    for i in range(len(cpallused)):
        tmp = []
        frused = struct.lattice.get_fractional_coords(cpallused[i])
        for j in range(len(frused)):
            tmp.append(list(frused[j]))
        frcpallused.append(modpos(tmp))

    for op in symm_ops:
        froperatedsites = []
        for i in range(len(sites)):
            frsites = struct.lattice.get_fractional_coords(sites[i])
            froperatedsites.append(list(op.operate(frsites)))
        froperatedsites.sort()
        modfropsites = modpos(froperatedsites)
        
        for used in frcpallused:
            if len(modfropsites) == len(used):
                if np.allclose(sorted(modfropsites), sorted(used), atol=0.01):
                    print('Symmetrically same structure found!')
                    return True

In [4]:
def getuniqueatoms(atoms, bareatoms, sites, maxmole, mindist, rused, group):
    '''
    Given surface Atom object and all possible attaching site positions, create all possible unique attached Atom object.
    Can exclude by the maximum number of molecules or minimum distance. 
    
    return
    allatoms   :all possible Atom object [[[x,y,z]], [[a,b,c]], [[x,y,z],[a,b,c]]]
    allused    :attached sites for each Atom object
    mindistlis :minimum distance of molecule for each Atom object
    numdict    :dictionary, key=number of attached molecule, value=number of object created
    '''
    allatoms = []
    allused = []
    molenum = []
    usedsites = []
    
    for i in range(len(rused)):
        tmp = []
        tmp.append(rused[i])
        allused.append(tmp)
        
    initallused = copy.deepcopy(allused)
    
    def recursive(ratoms, rsites, rused, molecules, tmpused):
        molecules += 1
        if molecules > maxmole:
            return None
        
        for i in range(len(rsites)):
            nextatoms = copy.deepcopy(ratoms)
            nextused = copy.deepcopy(rused)
            if molecules == 1:
                tmpused = copy.deepcopy(initallused)
                print('Used initialized!')
            
            add_adsorbate(nextatoms, molecule, h, rsites[i][:2])
            nextused.append(list(rsites[i]))
            
            dist = getmindist(nextused, bareatoms.cell)
            if dist < mindist:
                print('Distance {0} is below {1}'.format(dist, mindist))
                continue

            sameflag = checkifsame(bareatoms, nextused, tmpused)
            if sameflag:
                continue
            
            allatoms.append(nextatoms)
            allused.append(nextused)
            tmpused.append(nextused)
            molenum.append(molecules)
            struct = AseAtomsAdaptor.get_structure(nextatoms)
            
            print('{0}-------{1}-------'.format(molecules, nextatoms.symbols))
            
            try:
                nextsites = AdsorbateSiteFinder(struct).symm_reduce(sites)
                indexlist = []
                for j in range(len(nextused)):
                    for k in range(len(nextsites)):
                        if np.allclose(nextused[j], nextsites[k]):
                            indexlist.append(k)

                for j in range(len(usedsites)):
                    for k in range(len(nextsites)):
                        if np.allclose(usedsites[j], nextsites[k]):
                            indexlist.append(k)

                indexlist.sort(reverse = True) 
                for j in range(len(indexlist)):
                    nextsites.pop(indexlist[j])
                
                recursive(nextatoms, nextsites, nextused, molecules, tmpused)
            
            except:
                print('Error!!')
            
            if molecules == 1:
                for j in range(len(group)):
                    if rsites[i] in group[j]:
                        for k in range(len(group[j])):
                            usedsites.append(group[j][k])

    barestruct = AseAtomsAdaptor.get_structure(bareatoms)
    baresites = getadsites(bareatoms, True)
    redsites_ = baresites['all']
    redsites = [list(i) for i in redsites_]

    recursive(atoms, redsites, rused, len(rused), initallused)
    mindistlis = getmindistlist(allused, bareatoms.cell)
    
    numdict = {}
    for site in allused:
        if str(len(site)) not in numdict.keys():
            numdict[str(len(site))] = 1
        else:
            numdict[str(len(site))] += 1
    
    return allatoms, allused, mindistlis, numdict, molenum

In [5]:
def removemolecule(atoms, molecule):
    cpatoms = copy.deepcopy(atoms)
    poslis = []
    for i in reversed(range(len(cpatoms))):
        if cpatoms[i].symbol == molecule[0]:
            poslis.append(cpatoms[i].position[:2])

        for j in molecule:
            if cpatoms[i].symbol == j:
                cpatoms.pop(i)
                print(cpatoms.symbols)
                break
    return cpatoms, poslis

In [6]:
def creategroup(bareatoms, sites):
    barestruct = AseAtomsAdaptor.get_structure(bareatoms)
    baresites = getadsites(bareatoms, True)
    redsites_ = baresites['all']
    redsites = [list(i) for i in redsites_]

    unique = []
    for i in range(len(redsites)):
        tmp = []
        tmp.append(redsites[i])
        unique.append(tmp)
    
    group = copy.deepcopy(unique)
    num = 0
    for i in range(len(sites)):
        tmp = []
        tmp.append(sites[i])

        if tmp in unique:
            continue

        for j in range(len(unique)):
            tmp2 = []
            tmp2.append(unique[j])
            if checkifsame(bareatoms, tmp, tmp2):
                print(j, 'same config found!')
                group[j].append(sites[i])
                break

    return group

In [7]:
#############################################################################3
name = 'NiGa110_u1_no03_CO_n1_d3'
# name = 'NiGa_u1'
# name = 'Cu111_u2'
# name = 'NiGa110_u1_no03_CO_n1_d3'

inname = name + '.traj'
atoms = init_query(inname, 'spacom')
bareatoms, poslis = removemolecule(atoms, ['C', 'O'])

Ga2Ni2Ga2Ni2C
Ga2Ni2Ga2Ni2


In [8]:
barestruct = AseAtomsAdaptor.get_structure(bareatoms)
baresites = getadsites(bareatoms, False)
d = 1.13
h = 1.85
maxmole = 2
mindist = 0
molecule = Atoms('CO', positions=[(0., 0., 0.), (0., 0., d)], cell=(10., 10., 10.))

In [9]:
sites0_ = baresites['all']
sites0 = [list(i) for i in sites0_]

group = creategroup(bareatoms, sites0)

rused = []
for i in reversed(range(len(sites0))):
    for j in range(len(poslis)):
        if np.allclose(sites0[i][:2], poslis[j]):
            tmp = sites0.pop(i)
            rused.append(tmp)
baresites, rused

Symmetrically same structure found!
3 same config found!
Symmetrically same structure found!
2 same config found!
Symmetrically same structure found!
2 same config found!
Symmetrically same structure found!
2 same config found!
Symmetrically same structure found!
5 same config found!
Symmetrically same structure found!
6 same config found!
Symmetrically same structure found!
5 same config found!


({'ontop': [array([ 0.        ,  0.        , 18.24061245]),
   array([ 1.47092646,  2.08020415, 18.24061245])],
  'bridge': [array([1.47092646e+00, 9.23796217e-16, 1.82406124e+01]),
   array([ 0.73546323,  1.04010207, 18.24061245]),
   array([ 2.20638969,  1.04010207, 18.24061245]),
   array([1.30644514e-15, 2.08020415e+00, 1.82406124e+01]),
   array([ 0.73546323,  3.12030622, 18.24061245]),
   array([ 2.20638969,  3.12030622, 18.24061245])],
  'hollow': [array([ 1.47092646,  0.69340138, 18.24061245]),
   array([1.30644514e-15, 1.38680277e+00, 1.82406124e+01]),
   array([ 2.94185292,  2.77360553, 18.24061245]),
   array([ 1.47092646,  3.46700691, 18.24061245])],
  'all': [array([ 0.        ,  0.        , 18.24061245]),
   array([ 1.47092646,  2.08020415, 18.24061245]),
   array([ 0.73546323,  1.04010207, 18.24061245]),
   array([ 2.94185292,  2.08020415, 18.24061245]),
   array([ 2.20638969,  1.04010207, 18.24061245]),
   array([ 1.47092646,  0.        , 18.24061245]),
   array([ 0.735

In [10]:
allatoms, allused, mindistlis, numdict, molenum \
    = getuniqueatoms(atoms, bareatoms, sites0, maxmole, mindist, rused, group)

2-------Ga2Ni2Ga2Ni2COCO-------
2-------Ga2Ni2Ga2Ni2COCO-------
Error!!
2-------Ga2Ni2Ga2Ni2COCO-------
2-------Ga2Ni2Ga2Ni2COCO-------
2-------Ga2Ni2Ga2Ni2COCO-------
2-------Ga2Ni2Ga2Ni2COCO-------
2-------Ga2Ni2Ga2Ni2COCO-------


In [11]:
len(allatoms), numdict

(7, {'1': 1, '2': 7})

In [12]:
mindistlis

[2.94185291954,
 2.547719362519058,
 0.0,
 1.27385968125953,
 1.4709264597699985,
 2.0802041486601768,
 1.6261703870876274,
 1.3868027657734512]

In [13]:
count = 0
indexlist = []
for i in range(len(mindistlis)):
    if mindistlis[i] > 2.5:
        indexlist.append(i)
        count += 1
# count, indexlist

In [14]:
for i in range(len(allatoms)): # len(allatoms)
#     if 'COCOCOCO' in str(allatoms[i].symbols):
    view(allatoms[i])
    print(allatoms[i].symbols)
#         print('a')

Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO
Ga2Ni2Ga2Ni2COCO


In [15]:
for i in range(len(allatoms)):
    outname = name + str('_no') + str('{0:03d}'.format(i+1)) +'_CO_n' + str(molenum[i]) + str('_d') + str(int(np.ceil(mindistlis[i]/0.5)-3))  + '.traj'
    print(outname)
    outpath = '/home/katsuyut/init/' + str(outname)
#     allatoms[i].write(outpath)

NiGa110_u1_no03_CO_n1_d3_no001_CO_n2_d3.traj
NiGa110_u1_no03_CO_n1_d3_no002_CO_n2_d3.traj
NiGa110_u1_no03_CO_n1_d3_no003_CO_n2_d-3.traj
NiGa110_u1_no03_CO_n1_d3_no004_CO_n2_d0.traj
NiGa110_u1_no03_CO_n1_d3_no005_CO_n2_d0.traj
NiGa110_u1_no03_CO_n1_d3_no006_CO_n2_d2.traj
NiGa110_u1_no03_CO_n1_d3_no007_CO_n2_d1.traj


In [143]:
# for i in range(len(allatoms)):
#     if 'COCOCOCOCO' in str(allatoms[i].symbols):
#         outname = name + str('_no') + str('{0:02d}'.format(i+1)) +'_CO_n' + str(molenum[i]) + str('_d') + str(int(np.ceil(mindistlis[i]/0.5)-3))  + '.traj'
#         print(outname)
#         outpath = '/home/katsuyut/init/' + str(outname)
# #         allatoms[i].write(outpath)

NiGa110_u2_no06_CO_n5_d3.traj
NiGa110_u2_no28_CO_n5_d3.traj
NiGa110_u2_no31_CO_n5_d3.traj
NiGa110_u2_no35_CO_n5_d3.traj
NiGa110_u2_no41_CO_n5_d3.traj
NiGa110_u2_no49_CO_n5_d3.traj
NiGa110_u2_no55_CO_n5_d3.traj
NiGa110_u2_no63_CO_n5_d3.traj
NiGa110_u2_no88_CO_n5_d3.traj
NiGa110_u2_no93_CO_n5_d3.traj
NiGa110_u2_no97_CO_n5_d3.traj
NiGa110_u2_no105_CO_n5_d3.traj
NiGa110_u2_no117_CO_n5_d3.traj
NiGa110_u2_no135_CO_n5_d3.traj
NiGa110_u2_no144_CO_n5_d3.traj
NiGa110_u2_no147_CO_n5_d3.traj
NiGa110_u2_no166_CO_n5_d3.traj
NiGa110_u2_no171_CO_n5_d3.traj
NiGa110_u2_no173_CO_n5_d3.traj
NiGa110_u2_no193_CO_n5_d3.traj
NiGa110_u2_no203_CO_n5_d3.traj
NiGa110_u2_no205_CO_n5_d3.traj
NiGa110_u2_no207_CO_n5_d3.traj
NiGa110_u2_no211_CO_n5_d3.traj
NiGa110_u2_no345_CO_n5_d3.traj
NiGa110_u2_no355_CO_n5_d3.traj
NiGa110_u2_no362_CO_n5_d3.traj
NiGa110_u2_no364_CO_n5_d3.traj
NiGa110_u2_no384_CO_n5_d3.traj
NiGa110_u2_no408_CO_n5_d3.traj
NiGa110_u2_no417_CO_n5_d3.traj
NiGa110_u2_no440_CO_n5_d3.traj
NiGa110_u2_no444_CO